In [ ]:
from astropy.io import ascii
from astropy.table import setdiff, Table
import matplotlib.pyplot as plt
import numpy as np

def parallax_selec(cat, parallax, arc_range, k, plot_dance = False):
    
    %run Crossmatch.ipynb
    %run ds9_region.ipynb
    
    x_axis = "Parallax"
    y_axis = "Number"
    
    cat = cat.filled(999)
    
    cat.remove_rows([cat[parallax] == 999])
    
    confirmed_members = ascii.read('/home/yobd/Documents/SourcesForAnalysis/final_confirmed_table.dat')
    
    confirmed_members.remove_rows([confirmed_members['RUWE'] > 1.4])
    
    confirmed_members['Plx'] = confirmed_members['Plx'].filled(999)
    
    confirmed_members.remove_rows([confirmed_members['Plx'] == 999])
    
    parallax_selec = Table.copy(cat)
    
    mean_confirmed_plx = np.nanmean(confirmed_members['Plx'][confirmed_members['Plx'] < 8])
    std_confirmed_plx = np.absolute(np.nanstd(confirmed_members['Plx'][confirmed_members['Plx'] < 8]))
    
    # Reduces the values which will be used to calculate the linear-fit to an interval around the mean
    parallax_selec[parallax][parallax_selec[parallax] < (mean_confirmed_plx - k * std_confirmed_plx)] = np.nan
    parallax_selec[parallax][parallax_selec[parallax] > (mean_confirmed_plx + k * std_confirmed_plx)] = np.nan
    
    # First removes the entries from ARR_Y which correspond to NaN values in ARR_X
    # The other way around, ARR_X would have less rows than ARR_Y and an error would occur
    parallax_selec = parallax_selec[np.isfinite(parallax_selec[parallax])]
    
    parallax_selec[parallax] = np.absolute(parallax_selec[parallax])
    
    print("Mean:", mean_confirmed_plx)
    
    print("Standard deviation:", std_confirmed_plx)
    
    distance_parallax = np.absolute(1000 / parallax_selec[parallax])
    distance_parallax = distance_parallax[np.isfinite(distance_parallax)]
    
    rejects = setdiff(cat, parallax_selec, keys = ['ra', 'dec'])
    
    if len(rejects) != 0:
        
        #ascii.write(rejects, '/home/yobd/Documents/Parallax/rejects.cat', format = 'csv', overwrite = True)
        
        create_ds9_region(rejects['ra'], rejects['dec'], '/home/yobd/Documents/Parallax/rejects.reg')
    
    else:
        
        rejected = Table()
    
    #ascii.write(parallax_selec, '/home/yobd/Documents/Parallax/accepted.cat', format = 'csv', overwrite = True)
    
    create_ds9_region(parallax_selec['ra'], parallax_selec['dec'], '/home/yobd/Documents/Parallax/accepted.reg')
    
    rejects = rejects.filled(999)
    
    rejects.remove_rows([rejects[parallax] == 999])
    
    bin_number_accepted = np.int(np.absolute(( np.amin(parallax_selec[parallax]) - np.amax(parallax_selec[parallax]) ) / 0.2))
    bin_number_rejected = np.int(np.absolute(( np.amin(rejects[parallax]) - np.amax(rejects[parallax]) ) / 0.13))
    bin_number_cra = np.int(np.absolute(( np.amin(confirmed_members['Plx']) - np.amax(confirmed_members['Plx']) ) / 0.25))
    
    plt.rc('xtick',labelsize = 20)
    plt.rc('ytick',labelsize = 20)
    plt.figure(1)
    n_confirmed, bins_confirmed, patches_confirmed = plt.hist(confirmed_members['Plx'], bins = bin_number_cra, histtype = 'barstacked', color = 'C2', label = 'CrA Members', alpha = 1)
    n, bins, patches = plt.hist(parallax_selec[parallax], bins = bin_number_accepted, histtype = 'barstacked', color = 'k', label = r'CMD+PM-Selected Sources', alpha = 0.6)
    n_rej, bins_rej, patches_rej = plt.hist(rejects[parallax], bins = bin_number_rejected, histtype = 'barstacked', color = 'k', alpha = 0.6)
    plt.axvline(x = mean_confirmed_plx - k * std_confirmed_plx, color = 'k', linestyle = '--')
    plt.axvline(x = mean_confirmed_plx + k * std_confirmed_plx, color = 'k', linestyle = '--')
    plt.xlabel(x_axis + " [mas]", fontsize = 25)
    plt.ylabel(y_axis, fontsize = 25)
    plt.legend(fontsize = 11, loc = 1)
    plt.show()
    
    # The mean error of the parallaxes given by Gaia EDR3 to be plotted
    
    plx_error_mean = np.nanmean(parallax_selec['e_Plx'])
    
    plt.figure(2)
    
    plt.scatter(parallax_selec[parallax], parallax_selec['I_MAG_SUBARU'], marker = '.', color = 'k', label = 'CMD+PM-Selected Sources')
    plt.scatter(rejects[parallax], rejects['I_MAG_SUBARU'], marker = '.', color = 'k')
    plt.plot([2, 2 + plx_error_mean], [14.2, 14.2], lw = 5)
    plt.axvline(x = mean_confirmed_plx - k * std_confirmed_plx, color = 'k', linestyle = '--')
    plt.axvline(x = mean_confirmed_plx + k * std_confirmed_plx, color = 'k', linestyle = '--')
    plt.xlabel(x_axis + " [mas]", fontsize = 25)
    plt.ylabel('I [mag]', fontsize = 25)
    #plt.legend(fontsize = 11, loc = 4, markerscale = 3)
    plt.show()
    
    return parallax_selec